In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


# Tic Tac Toe Coin

An outer puzzle for an inner [tic-tac-toe puzzle](tic-tac-toe.ipynb). The tic-tac-toe puzzle is the game engine used by the tic-tac-toe coin puzzle to create a new coin with the next state.

## Chialisp
- [tic-tac-toe-coin.clsp](tic-tac-toe-coin.clsp)
- [tic-tac-toe.clsp](tic-tac-toe.clsp)
- [tic-tac-toe.clib](tic-tac-toe.clib)
- [utils.clib](utils.clip)

## Driver Code
- [tic_tac_toe.py](tic_tac_toe.py)
 

In [2]:
from chia.types.blockchain_format.program import Program
from cdv.util.load_clvm import load_clvm
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text

from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.')

In [3]:
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

# player 1 plays position 4
position = 4
print(f'{board} {player} {position}')

solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
next_player = 2
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 0, 0, 0, 0, 0] 1 4
((73 1023) (51 0x61ea31e738d0715272ec711a30dd9c521f59299cf527bed6201f461b973cfb23 1023))
   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


In [4]:
player, next_player = next_player, player
position = 5
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 1, 0, 0, 0, 0] 2 5
((73 1023) (51 0x2144e6103c8a3bb2ad22325d25636496a6a4567a2712abda95dcfbf22b6ff5fd 1023))
   |   |   
---+---+---
   | x | o 
---+---+---
   |   |   


In [5]:
player, next_player = next_player, player
position = 1
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 1, 2, 0, 0, 0] 1 1
((73 1023) (51 0xa613d8bd6fbaaebd12344f14fa32f63a4d7b0b3c555df239c5e075bad25cbbae 1023))
   | x |   
---+---+---
   | x | o 
---+---+---
   |   |   


In [6]:
player, next_player = next_player, player
position = 3
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, 
    board, next_player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 1, 0, 0, 1, 2, 0, 0, 0] 2 3
((73 1023) (51 0x1017de67052aee97f7710acf26e9a7e7d188ba9560aa979fe1c1c4cd783ce952 1023))
   | x |   
---+---+---
 o | x | o 
---+---+---
   |   |   


In [7]:
player, next_player = next_player, player
position = 7
print(f'{board} {player} {position}')
solution = Program.to([1023, position, next_player])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))

board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
print(board_state)
tic_tac_toe.print_board(board)

[0, 1, 0, 2, 1, 2, 0, 0, 0] 1 7
((51 0x900ddeed -113) (51 0xcafef00d 1023))
1
   | x |   
---+---+---
 o | x | o 
---+---+---
   | x |   


## Simulate Unplayable Board

In [8]:
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    player
)

# player 1 plays position 4
position = 0
print(f'{board} {player} {position}')

next_player = 2
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[0, 0, 0, 0, 0, 0, 0, 0, 0] 1 0
((73 1023) (51 0xf375c99bdd9e4fff7ca714490a84827987130c60644a7fa1b151210f1d8e8ef5 1023))
 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   


In [9]:
player, next_player = next_player, player
position = 1
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 0, 0, 0, 0, 0, 0, 0, 0] 2 1
((73 1023) (51 0x9dabbda198f5a049b9d7ae2612342c4008c6280e0742e883abce7a761ce92dc9 1023))
 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   


In [10]:
player, next_player = next_player, player
position = 2
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 0, 0, 0, 0, 0, 0, 0] 1 2
((73 1023) (51 0xd984df67cea704495be0c996df1fc512f8aab21fe4cec214763285af186d41b4 1023))
 x | o | x 
---+---+---
   |   |   
---+---+---
   |   |   


In [11]:
player, next_player = next_player, player
position = 3
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 0, 0, 0, 0, 0, 0] 2 3
((73 1023) (51 0x7f41ec1ca43e59c1ac043d77acde3743d5243ce7ea11fa141dcc2a6257a3fabf 1023))
 x | o | x 
---+---+---
 o |   |   
---+---+---
   |   |   


In [12]:
player, next_player = next_player, player
position = 4
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 0, 0, 0, 0, 0] 1 4
((73 1023) (51 0x2e2d3810bbde0d2682f02611837d3b592fca2d98de677eb0653d42e0e6668faf 1023))
 x | o | x 
---+---+---
 o | x |   
---+---+---
   |   |   


In [13]:
player, next_player = next_player, player
position = 5
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 1, 0, 0, 0, 0] 2 5
((73 1023) (51 0x32b3b8500b627a0cf01092fc4d29f65a6c71fb2248caf0872cac00540ff2398d 1023))
 x | o | x 
---+---+---
 o | x | o 
---+---+---
   |   |   


In [14]:
player, next_player = next_player, player
position = 7
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 1, 2, 0, 0, 0] 1 7
((73 1023) (51 0x4dcf40c6b96216cad87920329f13caf524ec68aa072c91cb072e7e4f63dae0bf 1023))
 x | o | x 
---+---+---
 o | x | o 
---+---+---
   | x |   


In [15]:
player, next_player = next_player, player
position = 6
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))
coin_puzzle_hash = conditions.at("rfrf")

## verify puzzle
board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
tic_tac_toe.print_board(board)
curried_tic_tac_toe_puzzle, curried_tic_tac_toe_coin_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzles(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle,
    board, 
    next_player
)

# new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
assert curried_tic_tac_toe_coin_puzzle.get_tree_hash() == coin_puzzle_hash

[1, 2, 1, 2, 1, 2, 0, 1, 0] 2 6
((73 1023) (51 0xbf8eabf0dd72e04efa17cfc6c35c6b41f507df97e9b27ba0ac136d99d59c992a 1023))
 x | o | x 
---+---+---
 o | x | o 
---+---+---
 o | x |   


In [16]:
player, next_player = next_player, player
position = 8
print(f'{board} {player} {position}')
solution = Program.to([1023, position])
conditions = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(conditions))

[1, 2, 1, 2, 1, 2, 2, 1, 0] 1 8
((51 0x900ddeed -113))


## Get Board From Curried Coin Puzzle

In [17]:
from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

from clvm_tools.binutils import disassemble
board = [0, 1, 1, 0, 2, 0, 2, 0, 0]
player_one_hash = bytes.fromhex("cafef00d")
player_two_hash = bytes.fromhex("deadbeef")
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(
    tic_tac_toe_puzzle, board, player
)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    player_one_hash,
    player_two_hash,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

print(disassemble(curried_tic_tac_toe_coin_puzzle))

(a (q 2 (q 2 22 (c 2 (c 5 (c 11 (c 23 (c 47 (c -65 (c (a (i (= -10529 (q . 1)) (q 1 . 2) (q 1 . 1)) 1) (c (a 95 (c 383 ())) ()))))))))) (c (q (73 51 4 (c 8 (c 11 ())) (c (c 20 (c 5 (c 11 ()))) ())) ((c (q . 2) (c (c (q . 1) 5) (c (a 26 (c 2 (c 11 (q 1)))) ()))) 2 (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 26 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) (a (i (not (= () 639)) (q 4 (c 20 (q 0x900ddeed -113)) (a (i (= (q . -1) 639) () (q 4 (a (i (= (q . 1) 639) (q 4 20 (c 11 (c 95 ()))) (q 4 20 (c 23 (c 95 ())))) 1) ())) 1)) (q 2 28 (c 2 (c (a 30 (c 2 (c (a 18 (c 2 (c 5 (c (c 5 (c 11 (c 23 (c 47 (c (a 18 (c 2 (c 47 (c (c 1407 (c -65 ())) ())))) ()))))) ())))) ()))) (c 95 ()))))) 1) 2 (i (l 5) (q 11 (q . 2) (a 30 (c 2 (c 9 ()))) (a 30 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 2 (q 2 22 (c 2 (c 5 (c 11 (c 23 (c 47 (c -65 (c (a (i (= -10529 (q . 1)) (q 1 . 2) (q 1 . 1)) 1) (c (a 95 (c 383 ())) ()))))))))) (c (q (73 51 4 (c 8 (c 11 ())) (c (c 20 (c 5 (c 11 ()))) ())) ((c (q . 2) (c (c (q .

In [18]:
board_from_puzzle = curried_tic_tac_toe_coin_puzzle.at("rrfrrfrrfrrfrrfrfrrrfrfr")
print(disassemble(board_from_puzzle))
board_from_puzzle = tic_tac_toe.get_board_from_curried_tic_tac_toe_coin_puzzle(curried_tic_tac_toe_coin_puzzle)
tic_tac_toe.print_board(board_from_puzzle)

(() 1 1 () 2 () 2 () ())
   | x | x 
---+---+---
   | o |   
---+---+---
 o |   |   


In [19]:
from clvm.casts import int_to_bytes
board_from_puzzle = curried_tic_tac_toe_coin_puzzle.at("rrfrrfrrfrrfrrfrfrrrfrfr")
solution = Program.to([board_from_puzzle, int_to_bytes(2), int_to_bytes(0)])
result = tic_tac_toe_puzzle.run(solution)
print(disassemble(result))

(() (a 1 1 () 2 () 2 () ()))
